In [2]:
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
import pickle
import psycopg2 as pg
import pandas.io.sql as pd_sql


In [3]:
with open('../../data/pickles/29feat_formodeling.pickle', 'rb') as read_file:
    df = pickle.load(read_file)

In [8]:
df.head()

,Student_Count_Total,Zip_Mean_Income,Fully Accessible,Generally accessible,Student_Count_Asian_perc,Student_Count_Asian_Pacific_Islander_perc,Student_Count_Black_perc,Student_Count_English_Learners_perc,Student_Count_Ethnicity_Not_Available_perc,Student_Count_Hawaiian_Pacific_Islander_perc,Student_Count_Hispanic_perc,Student_Count_Low_Income_perc,Student_Count_Multi_perc,Student_Count_Native_American_perc,Student_Count_Other_Ethnicity_perc,Student_Count_Special_Ed_perc,Student_Count_White_perc,Classroom_Languages_count,Blue,Brown,Green,Orange,Pink,Purple,Red,Dress_Code_dummie,grades_offered_count,charter,gender_f_bn,log_grad_rate
0,1739,53353.0,0,1,0.002875,0.0,0.033353,0.190339,0.006325,0.0,0.945371,0.953997,0.001725,0.003450,0.0,0.161587,0.006901,2,1,0,0,0,1,0,1,0,4,0,0,2.949688
1,1706,57770.0,0,1,0.005862,0.0,0.031067,0.201055,0.007620,0.0,0.942556,0.626026,0.003517,0.003517,0.0,0.162368,0.005862,2,1,0,0,0,1,0,1,0,4,0,0,3.063391
2,742,53353.0,0,0,0.000000,0.0,0.032345,0.227763,0.000000,0.0,0.962264,0.970350,0.000000,0.001348,0.0,0.196765,0.004043,1,1,0,0,1,1,0,0,1,4,1,1,2.844909
3,745,57770.0,0,0,0.000000,0.0,0.018792,0.285906,0.001342,0.0,0.974497,0.930201,0.000000,0.002685,0.0,0.195973,0.002685,1,1,0,0,1,1,0,0,1,4,1,1,2.708050
4,199,53353.0,0,0,0.000000,0.0,0.773869,0.010050,0.000000,0.0,0.180905,0.874372,0.000000,0.015075,0.0,0.291457,0.030151,0,0,0,0,0,0,0,0,0,4,0,1,4.588024


# PostgreSQL 

The data for this project is stored in a PostgreSQL data base.  The code below shows how one would create the tables and populate them with data.

In [12]:
connection_args = {
    'host': 'localhost',  
    'dbname': 'cps',    
    'port': 5432
}

conn = pg.connect(**connection_args)

In [13]:
cur = conn.cursor()

In [14]:
cur.execute('''CREATE TABLE IF NOT EXISTS highschool (
                schoolID int(6) NOT NULL,
                shortName varchar(100) DEFAULT NULL,
                longName varchar(100) NOT NULL,
                schoolType varchar(20) DEFAULT NULL, 
                graduationRate1617 double precision DEFAULT NULL,
                graduationRate1718 double precision DEFAULT NULL''');

SyntaxError: syntax error at or near "("
LINE 2:                 schoolID int(6) NOT NULL,
                                    ^


In [10]:
df_for_database = df_full[['School_ID', 'Short_Name',
                          'Long_Name', 'School_Type']]
                

NameError: name 'df_full' is not defined

In [13]:
df_for_database.head()

,School_ID,Short_Name,Long_Name,School_Type
0,609764,JUAREZ HS,Benito Juarez Community Academy High School,Neighborhood
1,609764,JUAREZ HS,Benito Juarez Community Academy High School,NaN
2,400054,NOBLE - PRITZKER HS,Noble - Pritzker College Prep,Charter
3,400054,NOBLE - PRITZKER HS,Noble - Pritzker College Prep,NaN
4,609726,BROOKS HS,Gwendolyn Brooks College Preparatory Academy HS,Selective enrollment


In [14]:
df_nodupes = df_for_database.drop_duplicates('School_ID')

In [20]:
df_nodupes.to_csv('data/schoolsforsql.csv', index=False)

In [22]:
# df_grad_rates_1718 = df_full[['School_ID','Graduation_Rate_School','School_Year']]
# df_grad_rates_1718 = df_grad_rates_1718[df_grad_rates_1718.School_Year=='School Year 2017-2018']
# df_grad_rates_1718.drop(columns = 'School_Year', inplace=True)
# df_grad_rates_1718.to_csv('data/gradrates1718.csv', index=False)

In [11]:
df_grad_rates_1617 = df_full[['School_ID','Graduation_Rate_School','School_Year']]
df_grad_rates_1617 = df_grad_rates_1617[df_grad_rates_1617.School_Year=='School Year 2016-2017']
df_grad_rates_1617.drop(columns = 'School_Year', inplace=True)
df_grad_rates_1617.to_csv('data/gradrates1617.csv', index=False)

In [15]:
# cur.execute('''CREATE TABLE IF NOT EXISTS hs_201617 (
#                 schoolID int NOT NULL,
#                 gradrate double precision DEFAULT NULL)''');

In [8]:
#Import full csv file for upload of the raw data to postgres
df_1718 = pd.read_csv('data/Chicago_Public_Schools_-_School_Profile_Information_SY1718.csv')

In [ ]:
#create 201718 raw table
cur.execute('''CREATE TABLE IF NOT EXISTS hs_201718_raw (
         School_ID int NOT NULL,
         Legacy_Unit_ID int DEFAULT NULL,
         Finance_ID int NOT NULL,
         Short_Name varchar(1000) DEFAULT NULL,
         Long_Name varchar(1000) DEFAULT NULL,
         Primary_Category varchar(2) DEFAULT NULL,
         Is_High_School varchar(1) DEFAULT NULL,
         Is_Middle_School varchar(1) DEFAULT NULL,
         Is_Elementary_School varchar(1) DEFAULT NULL,
         Is_Pre_School varchar(1) DEFAULT NULL,
         Summary varchar(10000) DEFAULT NULL, 
         Administrator_Title varchar(50) DEFAULT NULL,
         Administrator varchar(1000) DEFAULT NULL,
         Secondary_Contact_Title varchar(10000) DEFAULT NULL,
         Secondary_Contact varchar(1000) DEFAULT NULL,
         Phone BIGINT DEFAULT NULL,
         Fax BIGINT DEFAULT NULL,
         CPS_School_Profile varchar(1000) DEFAULT NULL,
         Website varchar(1000) DEFAULT NULL,
         Facebook varchar(1000) DEFAULT NULL,
         Twitter varchar(1000) DEFAULT NULL,
         Youtube varchar(1000) DEFAULT NULL,
         Pinterest varchar(1000) DEFAULT NULL,
         Attendance_Boundaries varchar(1) DEFAULT NULL,
         Grades_Offered_All varchar(1000) DEFAULT NULL,
         Grades_Offered varchar(1000) DEFAULT NULL,
         Student_Count_Total INT DEFAULT NULL,
         Student_Count_Low_Income INT DEFAULT NULL,
         Student_Count_Special_Ed INT DEFAULT NULL,
         Student_Count_English_Learners INT DEFAULT NULL,
         Student_Count_Black INT DEFAULT NULL,
         Student_Count_Hispanic INT DEFAULT NULL,
         Student_Count_White INT DEFAULT NULL,
         Student_Count_Asian INT DEFAULT NULL,
         Student_Count_Native_American INT DEFAULT NULL,
         Student_Count_Other_Ethnicity INT DEFAULT NULL,
         Student_Count_Asian_Pacific_Islander INT DEFAULT NULL,
         Student_Count_Multi INT DEFAULT NULL,
         Student_Count_Hawaiian_Pacific_Islander INT DEFAULT NULL,
         Student_Count_Ethnicity_Not_Available INT DEFAULT NULL,
         Statistics_Description varchar(1000) DEFAULT NULL,
         Demographic_Description varchar(1000) DEFAULT NULL,
         Dress_Code varchar(1) DEFAULT NULL,
         PreK_School_Day varchar(1000) DEFAULT NULL,
         Kindergarten_School_Day varchar(1000) DEFAULT NULL,
         School_Hours varchar(1000) DEFAULT NULL,
         Freshman_Start_End_Time varchar(1000) DEFAULT NULL,
         After_School_Hours varchar(1000) DEFAULT NULL,
         Earliest_Drop_Off_Time varchar(1000) DEFAULT NULL,
         Classroom_Languages varchar(1000) DEFAULT NULL,
         Bilingual_Services varchar(1) DEFAULT NULL,
         Refugee_Services varchar(1) DEFAULT NULL,
         Title_1_Eligible varchar(1) DEFAULT NULL,
         PreSchool_Inclusive varchar(10) DEFAULT NULL,
         Preschool_Instructional varchar(1) DEFAULT NULL,
         Significantly_Modified varchar(1) DEFAULT NULL,
         Hard_Of_Hearing varchar(1) DEFAULT NULL,
         Visual_Impairments varchar(1) DEFAULT NULL,
         Transportation_Bus varchar(1000) DEFAULT NULL,
         Transportation_El varchar(1000) DEFAULT NULL,
         Transportation_Metra varchar(1000) DEFAULT NULL,
         Average_ACT_School INT DEFAULT NULL,
         Mean_ACT INT DEFAULT NULL,
         College_Enrollment_Rate_School FLOAT DEFAULT NULL,
         College_Enrollment_Rate_Mean FLOAT DEFAULT NULL,
         Graduation_Rate_School FLOAT DEFAULT NULL,
         Graduation_Rate_Mean FLOAT DEFAULT NULL,
         Overall_Rating varchar(1000) DEFAULT NULL,
         Rating_Status varchar(1000) DEFAULT NULL,
         Rating_Statement varchar(1000) DEFAULT NULL,
         Classification_Description varchar(1000) DEFAULT NULL,
         School_Year varchar(1000) DEFAULT NULL,
         Third_Contact_Title varchar(1000) DEFAULT NULL,
         Third_Contact_Name varchar(1000) DEFAULT NULL,
         Fourth_Contact_Title varchar(1000) DEFAULT NULL,
         Fourth_Contact_Name varchar(1000) DEFAULT NULL,
         Fifth_Contact_Title varchar(1000) DEFAULT NULL,
         Fifth_Contact_Name varchar(1000) DEFAULT NULL,
         Sixth_Contact_Title varchar(1000) DEFAULT NULL,
         Sixth_Contact_Name varchar(1000) DEFAULT NULL,
         Seventh_Contact_Title varchar(1000) DEFAULT NULL,
         Seventh_Contact_Name varchar(1000) DEFAULT NULL,
         Network varchar(1000) DEFAULT NULL,
         Is_GoCPS_Participant varchar(1) DEFAULT NULL,
         Is_GoCPS_PreK varchar(1) DEFAULT NULL,
         Is_GoCPS_Elementary varchar(1) DEFAULT NULL,
         Is_GoCPS_High_School varchar(1) DEFAULT NULL,
         Open_For_Enrollment_Date varchar(1000) DEFAULT NULL,
         Closed_For_Enrollment_Date varchar(1000) DEFAULT NULL,
         School_Latitude FLOAT DEFAULT NULL,
         School_Longitude FLOAT DEFAULT NULL,
         Location varchar(1000) DEFAULT NULL

                )''');

In [39]:
#check the differences in the lists
len(list(df_1617)),len(list(df_1718))

(91, 92)

In [38]:
#check for differences to make import easier
for _ in list(df_1718):
    if _ not in list(df_1617):
        print(_)

Network
Is_GoCPS_Participant
Is_GoCPS_PreK
Is_GoCPS_Elementary
Is_GoCPS_High_School
Open_For_Enrollment_Date
Closed_For_Enrollment_Date


In [ ]:
#after successful import of 1718, try 1617
df_1617 = pd.read_csv('data/Chicago_Public_Schools_-_School_Profile_Information_SY1617.csv')

In [44]:
df_1617.Zip

0      60707
1      60651
2      60608
3      60632
4      60618
5      60618
6      60656
7      60628
8      60634
9      60608
10     60608
11     60639
12     60628
13     60614
14     60617
15     60632
16     60612
17     60639
18     60651
19     60649
20     60626
21     60607
22     60641
23     60644
24     60628
25     60632
26     60623
27     60614
28     60659
29     60644
       ...  
631    60625
632    60629
633    60621
634    60634
635    60622
636    60623
637    60609
638    60643
639    60626
640    60616
641    60647
642    60632
643    60607
644    60636
645    60620
646    60628
647    60631
648    60615
649    60610
650    60644
651    60612
652    60657
653    60612
654    60621
655    60660
656    60619
657    60647
658    60612
659    60615
660    60638
Name: Zip, Length: 661, dtype: int64

In [43]:
print(list(df_1617))

['School_ID', 'Legacy_Unit_ID', 'Finance_ID', 'Short_Name', 'Long_Name', 'School_Type', 'Primary_Category', 'Is_High_School', 'Is_Middle_School', 'Is_Elementary_School', 'Is_Pre_School', 'Summary', 'Administrator_Title', 'Administrator', 'Secondary_Contact_Title', 'Secondary_Contact', 'Address', 'City', 'State', 'Zip', 'Phone', 'Fax', 'CPS_School_Profile', 'Website', 'Facebook', 'Twitter', 'Youtube', 'Pinterest', 'Attendance_Boundaries', 'Grades_Offered_All', 'Grades_Offered', 'Student_Count_Total', 'Student_Count_Low_Income', 'Student_Count_Special_Ed', 'Student_Count_English_Learners', 'Student_Count_Black', 'Student_Count_Hispanic', 'Student_Count_White', 'Student_Count_Asian', 'Student_Count_Native_American', 'Student_Count_Other_Ethnicity', 'Student_Count_Asian_Pacific_Islander', 'Student_Count_Multi', 'Student_Count_Hawaiian_Pacific_Islander', 'Student_Count_Ethnicity_Not_Available', 'Statistics_Description', 'Demographic_Description', 'ADA_Accessible', 'Dress_Code', 'PreK_School

In [45]:
cur.execute('''CREATE TABLE IF NOT EXISTS hs_201617_raw (
         School_ID int NOT NULL,
         Legacy_Unit_ID int DEFAULT NULL,
         Finance_ID int NOT NULL,
         Short_Name varchar(1000) DEFAULT NULL,
         Long_Name varchar(1000) DEFAULT NULL,
         School_Type varchar(1000) DEFAULT NULL,
         Primary_Category varchar(2) DEFAULT NULL,
         Is_High_School varchar(1) DEFAULT NULL,
         Is_Middle_School varchar(1) DEFAULT NULL,
         Is_Elementary_School varchar(1) DEFAULT NULL,
         Is_Pre_School varchar(1) DEFAULT NULL,
         Summary varchar(10000) DEFAULT NULL, 
         Administrator_Title varchar(50) DEFAULT NULL,
         Administrator varchar(1000) DEFAULT NULL,
         Secondary_Contact_Title varchar(10000) DEFAULT NULL,
         Secondary_Contact varchar(1000) DEFAULT NULL,
         Address varchar(1000) DEFAULT NULL,
         City varchar(1000) DEFAULT NULL,
         State varchar(100) DEFAULT NULL,
         Zip INT DEFAULT NULL,
         Phone BIGINT DEFAULT NULL,
         Fax BIGINT DEFAULT NULL,
         CPS_School_Profile varchar(1000) DEFAULT NULL,
         Website varchar(1000) DEFAULT NULL,
         Facebook varchar(1000) DEFAULT NULL,
         Twitter varchar(1000) DEFAULT NULL,
         Youtube varchar(1000) DEFAULT NULL,
         Pinterest varchar(1000) DEFAULT NULL,
         Attendance_Boundaries varchar(1) DEFAULT NULL,
         Grades_Offered_All varchar(1000) DEFAULT NULL,
         Grades_Offered varchar(1000) DEFAULT NULL,
         Student_Count_Total INT DEFAULT NULL,
         Student_Count_Low_Income INT DEFAULT NULL,
         Student_Count_Special_Ed INT DEFAULT NULL,
         Student_Count_English_Learners INT DEFAULT NULL,
         Student_Count_Black INT DEFAULT NULL,
         Student_Count_Hispanic INT DEFAULT NULL,
         Student_Count_White INT DEFAULT NULL,
         Student_Count_Asian INT DEFAULT NULL,
         Student_Count_Native_American INT DEFAULT NULL,
         Student_Count_Other_Ethnicity INT DEFAULT NULL,
         Student_Count_Asian_Pacific_Islander INT DEFAULT NULL,
         Student_Count_Multi INT DEFAULT NULL,
         Student_Count_Hawaiian_Pacific_Islander INT DEFAULT NULL,
         Student_Count_Ethnicity_Not_Available INT DEFAULT NULL,
         Statistics_Description varchar(1000) DEFAULT NULL,
         Demographic_Description varchar(1000) DEFAULT NULL,
         ADA_Accessible varchar(1000) DEFAULT NULL,
         Dress_Code varchar(1) DEFAULT NULL,
         PreK_School_Day varchar(1000) DEFAULT NULL,
         Kindergarten_School_Day varchar(1000) DEFAULT NULL,
         School_Hours varchar(1000) DEFAULT NULL,
         Freshman_Start_End_Time varchar(1000) DEFAULT NULL,
         After_School_Hours varchar(1000) DEFAULT NULL,
         Earliest_Drop_Off_Time varchar(1000) DEFAULT NULL,
         Classroom_Languages varchar(1000) DEFAULT NULL,
         Bilingual_Services varchar(1) DEFAULT NULL,
         Refugee_Services varchar(1) DEFAULT NULL,
         Title_1_Eligible varchar(1) DEFAULT NULL,
         PreSchool_Inclusive varchar(10) DEFAULT NULL,
         Preschool_Instructional varchar(1) DEFAULT NULL,
         Significantly_Modified varchar(1) DEFAULT NULL,
         Hard_Of_Hearing varchar(1) DEFAULT NULL,
         Visual_Impairments varchar(1) DEFAULT NULL,
         Transportation_Bus varchar(1000) DEFAULT NULL,
         Transportation_El varchar(1000) DEFAULT NULL,
         Transportation_Metra varchar(1000) DEFAULT NULL,
         School_Latitude FLOAT DEFAULT NULL,
         School_Longitude FLOAT DEFAULT NULL,
         Average_ACT_School FLOAT DEFAULT NULL,
         Mean_ACT FLOAT DEFAULT NULL,
         College_Enrollment_Rate_School FLOAT DEFAULT NULL,
         College_Enrollment_Rate_Mean FLOAT DEFAULT NULL,
         Graduation_Rate_School FLOAT DEFAULT NULL,
         Graduation_Rate_Mean FLOAT DEFAULT NULL,
         Overall_Rating varchar(1000) DEFAULT NULL,
         Rating_Status varchar(1000) DEFAULT NULL,
         Rating_Statement varchar(1000) DEFAULT NULL,
         Classification_Description varchar(1000) DEFAULT NULL,
         School_Year varchar(1000) DEFAULT NULL,
         Third_Contact_Title varchar(1000) DEFAULT NULL,
         Third_Contact_Name varchar(1000) DEFAULT NULL,
         Fourth_Contact_Title varchar(1000) DEFAULT NULL,
         Fourth_Contact_Name varchar(1000) DEFAULT NULL,
         Fifth_Contact_Title varchar(1000) DEFAULT NULL,
         Fifth_Contact_Name varchar(1000) DEFAULT NULL,
         Sixth_Contact_Title varchar(1000) DEFAULT NULL,
         Sixth_Contact_Name varchar(1000) DEFAULT NULL,
         Seventh_Contact_Title varchar(1000) DEFAULT NULL,
         Seventh_Contact_Name varchar(1000) DEFAULT NULL,
         Location varchar(1000) DEFAULT NULL

                )''');

In [46]:
conn.commit()

In [13]:
cur.execute('''CREATE TABLE IF NOT EXISTS hs_modeldata (
         school_id int not null,
         school_year varchar(1000) not null,
         student_count_total int default null,
         fully_accessible int NOT NULL,
         generally_accessible INT NOT NULL,
         student_count_asian_perc FLOAT default null,
         student_count_asian_pacific_islander_perc FLOAT default null,
         student_count_black_perc FLOAT default null,
         student_count_english_learners_perc FLOAT default null,
         student_count_ethnicity_not_available_perc FLOAT default null,
         student_count_hawaiian_pacific_islander_perc FLOAT default null,
         student_count_hispanic_perc FLOAT default null,
         student_count_low_income_perc FLOAT default null,
         student_count_multi_perc FLOAT default null,
         student_count_native_american_perc FLOAT default null,
         student_count_other_ethnicity_perc FLOAT default null,
         student_count_special_ed_perc FLOAT default null,
         student_count_white_perc FLOAT default null,
         classroom_langauges_count INT DEFAULT NULL,
         blue INT DEFAULT NULL,
         brown INT DEFAULT NULL,
         green INT DEFAULT NULL,
         orange INT DEFAULT NULL,
         pink INT DEFAULT NULL,
         purple INT DEFAULT NULL,
         red INT DEFAULT NULL,
         dress_code_dummie INT default null,
         grades_offered_count INT default null,
         charter INT DEFAULT NULL,
         gender_f_bn INT DEFAULT NULL,
         log_grade_rate FLOAT DEFAULT NULL,
         zip FLOAT DEFAULT NULL,
         zip_mean_income FLOAT DEFAULT NULL
                )''');


In [14]:
conn.commit()